In [1]:
from google.colab import drive
drive.mount('/content/drive/')
import sys
sys.path.append('/content/drive/MyDrive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import os
import h5py
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array


# Directory where the images and density maps are stored
data_dir = "/content/drive/MyDrive/grape"

# Get a list of filenames for the input images and density maps
image_filenames = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.jpg')]
density_map_filenames = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.h5')]

# Load the images and their corresponding density maps
images = []
density_maps = []
for image_filename, density_map_filename in zip(image_filenames, density_map_filenames):
    # Load the image and resize to (224,224)
    image = Image.open(image_filename)
    image = image.convert('RGB')
    image = image.resize((224,224))
    
    # Convert the image to a numpy array
    x = img_to_array(image)
    # Preprocess the input by subtracting the mean RGB pixel intensity of the ImageNet dataset
    x = tf.keras.applications.vgg16.preprocess_input(x)
    images.append(x)

    # Load the density map and resize to (7,7,1)
    with h5py.File(density_map_filename, 'r') as f:
        density_map = np.array(f['density'])
        density_map = Image.fromarray(density_map)
        # Resize density map to (7,7)
        density_map = np.array(density_map)
        density_map = cv2.resize(density_map, (7,7))
        # Reshape density map to (7,7,1)
        density_map = np.reshape(density_map, (7,7,1))
        #density_map = np.array(density_map)
        density_maps.append(density_map)


In [3]:
features=np.array(images)
target=np.array(density_maps)

In [4]:
features_float = features.astype('float32') # convert to float32
target_float = target.astype('float32') # convert to float32

In [5]:
from sklearn.model_selection import train_test_split

# Split the data into training, validation, and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_float,target_float, test_size=0.2, random_state=42)

# Print the sizes of the resulting arrays
# print("X_train shape:", X_train.shape)
# print("y_train shape:", y_train.shape)
# print("X_test shape:", X_test.shape)
# print("y_test shape:", y_test.shape)


In [6]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Dense, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [7]:
import tensorflow as tf
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Reshape, Activation
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.optimizers import Adam

# Reshape X_train and y_train to match the input and output shape of the model
X_train = tf.reshape(X_train, (-1, X_train.shape[1], X_train.shape[2], 3))
y_train = tf.reshape(y_train, (-1, y_train.shape[1], y_train.shape[2], 1))

# Define the GBCNet model
vgg = VGG16(weights='imagenet', include_top=False, input_shape=X_train[0].shape)

for layer in vgg.layers:
    layer.trainable = False

input_layer = vgg.input
block5_pool = vgg.get_layer('block5_pool').output

conv1 = Conv2D(512, (3,3), dilation_rate=(2,2), padding='same', activation='relu')(block5_pool)
conv2 = Conv2D(512, (3,3), dilation_rate=(2,2), padding='same', activation='relu')(conv1)
conv3 = Conv2D(512, (3,3), dilation_rate=(2,2), padding='same', activation='relu')(conv2)
conv4 = Conv2D(256, (3,3), dilation_rate=(2,2), padding='same', activation='relu')(conv3)
conv5 = Conv2D(128, (3,3), dilation_rate=(2,2), padding='same', activation='relu')(conv4)
conv6 = Conv2D(64, (3,3), dilation_rate=(2,2), padding='same', activation='relu')(conv5)
conv7 = Conv2D(1, (3,3), dilation_rate=(2,2), padding='same', activation='sigmoid')(conv6)

# Remove the UpSampling2D layer and the second Activation layer, and add a Reshape layer to reshape the output to (7,7,1)
output = Reshape((7,7,1))(conv7)

model = Model(inputs=input_layer, outputs=output)

# Compile and train the model
model.compile(loss='MSE', optimizer=Adam(learning_rate=1e-4))

model.fit(X_train, y_train, batch_size=16, epochs=10)


Epoch 1/10
1/1 [==============================] - 10s 10s/step - loss: 0.2471
Epoch 2/10
1/1 [==============================] - 10s 10s/step - loss: 0.0562
Epoch 3/10
1/1 [==============================] - 8s 8s/step - loss: 0.0139
Epoch 4/10
1/1 [==============================] - 15s 15s/step - loss: 0.0026
Epoch 5/10
1/1 [==============================] - 14s 14s/step - loss: 3.8380e-04
Epoch 6/10
1/1 [==============================] - 10s 10s/step - loss: 5.5069e-05
Epoch 7/10
1/1 [==============================] - 8s 8s/step - loss: 9.9692e-06
Epoch 8/10
1/1 [==============================] - 11s 11s/step - loss: 3.8431e-06
Epoch 9/10
1/1 [==============================] - 10s 10s/step - loss: 2.9964e-06
Epoch 10/10
1/1 [==============================] - 8s 8s/step - loss: 2.8793e-06


In [8]:
# Evaluate the model
y_pred = model.predict(X_test)
y_pred = np.reshape(y_pred, (4, 7, 7))

y_test = y_test.reshape((y_test.shape[0], -1))

mae = mean_absolute_error(y_test, y_pred.reshape((y_pred.shape[0], -1)))
mse = mean_squared_error(y_test, y_pred.reshape((y_pred.shape[0], -1)))

print("Mean Absolute Error: ", mae)
print("Mean Squared Error: ", mse)

1/1 [==============================] - 4s 4s/step
Mean Absolute Error:  0.00059596193
Mean Squared Error:  3.1300915e-06
